In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
from glob import glob
import random
import time
import tensorflow
import datetime
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 3 = INFO, WARNING, and ERROR messages are not printed

from tqdm import tqdm

import numpy as np
import pandas as pd
from IPython.display import FileLink
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns 
%matplotlib inline
from IPython.display import display, Image
import matplotlib.image as mpimg
import cv2

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files       
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.metrics import log_loss

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.vgg16 import VGG16

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

#importing all the necessary modules/packages for this project
print("modules imported")

modules imported


In [24]:
trainPath = 'C:/Users/Sarwar/Downloads/Stanford40/JPEGImages' #where the image set is located
IMG_SIZE = 224
batch_size = 300
nb_epoch = 1

In [17]:
def label_img(img): #this labels the images
    word_label = img.split('_')[0]
    next_word = img.split('_')[1]
    if len(img.split('_')) > 2:
        word_3 = img.split('_')[2]
    else:
        word_3 = 'nada' 
    if len(img.split('_')) > 3:
        word_4 = img.split('_')[3]
    else:
        word_4 = 'nada'
        
    label = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    i = 0
    # 40-labels conversion ladder 
    if word_label == 'applauding': i = 0
    elif word_label == 'blowing': i = 1
    elif word_label == 'brushing': i = 2
    elif word_label == 'cleaning': i = 3
    elif word_label == 'climbing': i = 4
    elif word_label == 'cooking': i = 5
    elif word_label == 'cutting' and next_word == 'trees': i = 6
    elif word_label == 'cutting': i = 7
    elif word_label == 'drinking': i = 8
    elif word_label == 'feeding': i = 9
    #10
    elif word_label == 'fishing': i = 10
    elif word_label == 'fixing' and word_3 == 'bike': i = 11
    elif word_label == 'fixing': i = 12
    elif word_label == 'gardening': i = 13
    elif word_label == 'holding': i = 14
    elif word_label == 'jumping': i = 15
    elif word_label == 'looking' and word_4 == 'microscope': i = 16
    elif word_label == 'looking': i = 17
    elif word_label == 'playing' and next_word == 'guitar': i = 18
    elif word_label == 'playing': i = 19
    #20
    elif word_label == 'pouring': i = 20
    elif word_label == 'pushing': i = 21
    elif word_label == 'reading': i = 22
    elif word_label == 'phoning': i = 23
    elif word_label == 'riding' and word_3 == 'bike': i = 24
    elif word_label == 'riding': i = 25
    elif word_label == 'rowing': i = 26
    elif word_label == 'running': i = 27
    elif word_label == 'shooting': i = 28
    elif word_label == 'smoking': i = 29
    #30
    elif word_label == 'taking': i = 30
    elif word_label == 'texting': i = 31
    elif word_label == 'throwing': i = 32
    elif word_label == 'using': i = 33
    elif word_label == 'walking': i = 34
    elif word_label == 'washing': i = 35
    elif word_label == 'watching': i = 36
    elif word_label == 'waving': i = 37
    elif word_label == 'writing' and word_4 == 'board': i = 38
    elif word_label == 'writing': i = 39
    #40
    label[i] = 1
    return label

In [32]:
def create_train_data_x(): #creates the x-variable of the training data
    training_data_x = []
    for img in tqdm(os.listdir(trainPath)):
        #label = label_img(img)
        path = os.path.join(trainPath,img)
        img = cv2.imread(path)
        img = tensorflow.keras.preprocessing.image.load_img(path, target_size=[IMG_SIZE, IMG_SIZE])
        training_data_x.append([np.array(img)])
    np.save('train_data_x.npy', training_data_x)
    return training_data_x

def create_train_data_y(): #creates the y-variable of the training data
    training_data_y = []
    for img in tqdm(os.listdir(trainPath)):
        label = label_img(img)
        #path = os.path.join(trainPath,img)
        #img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data_y.append([label])
    np.save('train_data_y.npy', training_data_y)
    return training_data_y

#def process_test_data():
#lol

In [33]:
#testing my $H!T works so far
txt = "waving_hands_182.jpg"
label_img(txt)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0]

In [34]:
def create_model():
    return tensorflow.keras.applications.MobileNet()

In [35]:
#DON'T TOUCH DIS ONE BOY
x_train, y_train = create_train_data_x(), create_train_data_y()
# If you have already created the dataset:
x_train, y_train = np.load('train_data_x.npy'), np.load('train_data_y.npy')
X = np.array(x_train).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [y_train]

100%|█████████████████████████████████████████████████████████████████████████████| 9532/9532 [00:43<00:00, 220.83it/s]


In [36]:
model = create_model()
model.compile(optimizer='adam',  # Good default optimizer to start with
              loss='sparse_categorical_crossentropy',  # how will we calculate our "error." Neural network aims to minimize loss.
              metrics=['accuracy'])  # what to track
model.fit(X, Y, epochs=nb_epoch)  # train the model
'''
model.save('action_stanford.model')
new_model = tf.keras.models.load_model('action_stanford.model')
predictions = new_model.predict(x_test)
print(predictions)
print(np.argmax(predictions[0]))
plt.imshow(x_test[0], cmap=plt.cm.binary)
plt.show()
'''

ValueError: Error when checking input: expected input_7 to have shape (224, 224, 3) but got array with shape (224, 224, 1)